## Steps

1. Get metadata using script: extraction_slm.py, date of metadata extraction: 6/11/24
2. This notebook to filter according to following criteria:
  - humaneval
  - num_parameters <5B
  - has safetensors
3. Obtained dataset with 403? possible models
4. Check manually to those which need permission
5. Obtain random models
- Considering 1 per organization

- Execute from "Read file section"
- (optional) stratified sampling, if many models

- Manual selection:
  - discard any fine-tunning or quantized model, checked on model name and checked on model card
  - discard those that do not have explicitly HumanEval (some we considered because they may have MultiHumanEval )
###  first selection

- text-generation []
- decoder []

- open-source models [x]
- HF availability [x]
- Task: text generation [x]
- Date []
- architecture: decoder-based only,  []
- model size < SLMs size [x]
  - https://huggingface.co/docs/accelerate/main/en/usage_guides/model_size_estimator?
  - model size from endpoint?
  - safetensors -> model_info.safetensors -> https://huggingface.co/microsoft/phi-2 [x]
  - check it matches our data about models used []

- [?]Code generation [] used in code generation, trained in code datasets?


- 

In [3]:
from huggingface_hub import HfApi
import pandas as pd


In [4]:
pwd

'/home/fjdur/energy-ml-serving'

In [5]:
import sys
sys.path.append("model_selection/") 

In [6]:
save_dir = 'model_selection/tmp/'

## Read dataset from extraction_slm.py 

In [53]:
import math
import re

In [141]:
def check_model(model_name, df):
    return model_name in df['model'].values

In [165]:
file_name = 'model_selection/results/decoder_models_info_02.csv'
df = pd.read_csv(file_name)
print(f'total models: {len(df)}')

total models: 104816


In [166]:

check_model("microsoft/Phi-3-mini-128k-instruct" ,df)

True

In [167]:
df.iloc[0]['safetensors']
df.iloc[0]['eval_results']


nan

In [168]:
df.columns

Index(['model', 'id', 'created_at', 'downloads_all_time', 'downloads', 'likes',
       'safetensors', 'task', 'datasets', 'eval_results', 'metrics',
       'library_name', 'transformers_info', 'architecture', 'tags',
       'card_data'],
      dtype='object')

In [169]:
# Filter out rows where the 'safetensors' column is NaN
filtered_df = df[df['safetensors'].notna()]

# Display the filtered DataFrame
print("After safetensors check: ",len(filtered_df))

After safetensors check:  67761


In [170]:
# feature: Adding num_parameters column

# Function to extract the 'total' value from the 'safetensors' string
def extract_total(safetensors_info):
    if pd.isna(safetensors_info):
        return None
    match = re.search(r"total=(\d+)", safetensors_info)
    return int(match.group(1)) if match else None

# Create a new column 'num_parameters' with the extracted total
filtered_df['num_parameters'] = filtered_df['safetensors'].apply(extract_total)
filtered_df.head()

/tmp/ipykernel_180107/2495092620.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['num_parameters'] = filtered_df['safetensors'].apply(extract_total)


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,metrics,library_name,transformers_info,architecture,tags,card_data,num_parameters
0,HuggingFaceTB/SmolLM2-1.7B-Instruct,HuggingFaceTB/SmolLM2-1.7B-Instruct,2024-10-31 13:42:06+00:00,Unknown,16940,233,SafeTensorsInfo(parameters={'BF16': 1711376384...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'tensorboard', 'onnx', 'safet...",language:\n- en\nlicense: apache-2.0\nlibrary_...,1711376384
1,nvidia/Llama-3.1-Nemotron-70B-Instruct-HF,nvidia/Llama-3.1-Nemotron-70B-Instruct-HF,2024-10-12 02:37:13+00:00,Unknown,186569,1495,SafeTensorsInfo(parameters={'BF16': 7055370649...,text-generation,['nvidia/HelpSteer2'],NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\nlicense: llama3.1\nlibrary_na...,70553706496
2,facebook/MobileLLM-1B,facebook/MobileLLM-1B,2024-10-31 00:07:47+00:00,Unknown,3770,92,SafeTensorsInfo(parameters={'BF16': 1046421760...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",license: cc-by-nc-4.0\nlibrary_name: transformers,1046421760
3,meta-llama/Llama-3.2-1B,meta-llama/Llama-3.2-1B,2024-09-18 15:03:14+00:00,Unknown,1293208,797,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,1235814400
4,facebook/MobileLLM-125M,facebook/MobileLLM-125M,2024-10-30 22:48:34+00:00,Unknown,3406,70,"SafeTensorsInfo(parameters={'F16': 124635456},...",text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",license: cc-by-nc-4.0\nlibrary_name: transformers,124635456


In [171]:
# Filter out rows where the 'safetensors' column is NaN
limit_slm_parameters=5 *1e9 # 10B
print(len(filtered_df))
print(f'limit of num_parameters considered: {limit_slm_parameters}')
filtered_df = filtered_df[filtered_df['num_parameters'] <= limit_slm_parameters]

# Display the filtered DataFrame
print(len(filtered_df))
filtered_df.head()

67761
limit of num_parameters considered: 5000000000.0
32242


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,metrics,library_name,transformers_info,architecture,tags,card_data,num_parameters
0,HuggingFaceTB/SmolLM2-1.7B-Instruct,HuggingFaceTB/SmolLM2-1.7B-Instruct,2024-10-31 13:42:06+00:00,Unknown,16940,233,SafeTensorsInfo(parameters={'BF16': 1711376384...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'tensorboard', 'onnx', 'safet...",language:\n- en\nlicense: apache-2.0\nlibrary_...,1711376384
2,facebook/MobileLLM-1B,facebook/MobileLLM-1B,2024-10-31 00:07:47+00:00,Unknown,3770,92,SafeTensorsInfo(parameters={'BF16': 1046421760...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",license: cc-by-nc-4.0\nlibrary_name: transformers,1046421760
3,meta-llama/Llama-3.2-1B,meta-llama/Llama-3.2-1B,2024-09-18 15:03:14+00:00,Unknown,1293208,797,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,1235814400
4,facebook/MobileLLM-125M,facebook/MobileLLM-125M,2024-10-30 22:48:34+00:00,Unknown,3406,70,"SafeTensorsInfo(parameters={'F16': 124635456},...",text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",license: cc-by-nc-4.0\nlibrary_name: transformers,124635456
5,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-3.2-3B-Instruct,2024-09-18 15:19:20+00:00,Unknown,1041938,523,SafeTensorsInfo(parameters={'BF16': 3212749824...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,3212749824


In [172]:
# feature: Adding popularity column: sum of normalized likes and downloads

# Normalize the 'likes' and 'downloads' columns
#filtered_df['normalized_likes'] = df['likes'] / df['likes'].max()
#df['normalized_downloads'] = df['downloads'] / df['downloads'].max()

# Calculate the sum of the normalized 'likes' and 'downloads'
print(f"likes max: {filtered_df['likes'].max()} downloads max: {filtered_df['downloads'].max()}")
filtered_df['popularity'] = (filtered_df['likes'] / filtered_df['likes'].max()) + (filtered_df['downloads'] / filtered_df['downloads'].max())
print(f"popularity max: {filtered_df['popularity'].max()} ")


filtered_df.head()

likes max: 3239 downloads max: 15341589
popularity max: 1.7246063599876504 


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,metrics,library_name,transformers_info,architecture,tags,card_data,num_parameters,popularity
0,HuggingFaceTB/SmolLM2-1.7B-Instruct,HuggingFaceTB/SmolLM2-1.7B-Instruct,2024-10-31 13:42:06+00:00,Unknown,16940,233,SafeTensorsInfo(parameters={'BF16': 1711376384...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'tensorboard', 'onnx', 'safet...",language:\n- en\nlicense: apache-2.0\nlibrary_...,1711376384,0.073040
2,facebook/MobileLLM-1B,facebook/MobileLLM-1B,2024-10-31 00:07:47+00:00,Unknown,3770,92,SafeTensorsInfo(parameters={'BF16': 1046421760...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",license: cc-by-nc-4.0\nlibrary_name: transformers,1046421760,0.028650
3,meta-llama/Llama-3.2-1B,meta-llama/Llama-3.2-1B,2024-09-18 15:03:14+00:00,Unknown,1293208,797,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,1235814400,0.330358
4,facebook/MobileLLM-125M,facebook/MobileLLM-125M,2024-10-30 22:48:34+00:00,Unknown,3406,70,"SafeTensorsInfo(parameters={'F16': 124635456},...",text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",license: cc-by-nc-4.0\nlibrary_name: transformers,124635456,0.021834
5,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-3.2-3B-Instruct,2024-09-18 15:19:20+00:00,Unknown,1041938,523,SafeTensorsInfo(parameters={'BF16': 3212749824...,text-generation,NaN,NaN,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,3212749824,0.229386


In [150]:
# # Filter rows where 'card_data' contains the string 'humaneval' (case-insensitive)
# df_test = filtered_df[
#     filtered_df['card_data'].str.contains('humaneval', case=False, na=False)
# ]

# # Display the length of the filtered DataFrame
# print("Filtered DataFrame Length:", len(df_test))

In [173]:
print(check_model("microsoft/Phi-3-mini-128k-instruct" ,filtered_df))


True


In [174]:
model_name = "microsoft/Phi-3-mini-128k-instruct" 

In [175]:
print(len(filtered_df))

32242


In [176]:

filtered_df['humaneval_in_card'] = filtered_df['card_data'].str.contains('humaneval', case=False, na=False)

filtered_df['humaneval_in_eval_results'] = filtered_df['eval_results'].str.contains('humaneval', case=False, na=False)

filtered_df['need_permission'] = filtered_df['card_data'].str.contains('extra_gated', case=False, na=False)

In [177]:
gemma = filtered_df[filtered_df['model'] == model_name]

gemma

,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,...,library_name,transformers_info,architecture,tags,card_data,num_parameters,popularity,humaneval_in_card,humaneval_in_eval_results,need_permission
79,microsoft/Phi-3-mini-128k-instruct,microsoft/Phi-3-mini-128k-instruct,2024-04-22 16:26:23+00:00,Unknown,529382,1598,SafeTensorsInfo(parameters={'BF16': 3821079552...,text-generation,NaN,NaN,...,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Phi3ForCausalLM'],"['transformers', 'safetensors', 'phi3', 'text-...",language:\n- en\nlicense: mit\ntags:\n- nlp\n-...,3821079552,0.527868,False,False,False


In [156]:
# Filter rows where any of the specified columns have a True value
filtered_df = filtered_df[
    filtered_df[['humaneval_in_card', 'humaneval_in_eval_results', 'need_permission']].any(axis=1)
]

# Display the filtered DataFrame
print("Filtered DataFrame Length:", len(filtered_df))
filtered_df.head()


Filtered DataFrame Length: 430


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,...,library_name,transformers_info,architecture,tags,card_data,num_parameters,popularity,humaneval_in_card,humaneval_in_eval_results,need_permission
3,meta-llama/Llama-3.2-1B,meta-llama/Llama-3.2-1B,2024-09-18 15:03:14+00:00,Unknown,1293208,797,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,NaN,NaN,...,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,1235814400,0.330358,False,False,True
5,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-3.2-3B-Instruct,2024-09-18 15:19:20+00:00,Unknown,1041938,523,SafeTensorsInfo(parameters={'BF16': 3212749824...,text-generation,NaN,NaN,...,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,3212749824,0.229386,False,False,True
10,meta-llama/Llama-3.2-1B-Instruct,meta-llama/Llama-3.2-1B-Instruct,2024-09-18 15:12:47+00:00,Unknown,1318521,504,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,NaN,NaN,...,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,1235814400,0.241548,False,False,True
16,meta-llama/Llama-3.2-3B,meta-llama/Llama-3.2-3B,2024-09-18 15:23:48+00:00,Unknown,248764,292,SafeTensorsInfo(parameters={'BF16': 3212749824...,text-generation,NaN,NaN,...,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,3212749824,0.106366,False,False,True
34,google/gemma-2-2b,google/gemma-2-2b,2024-07-16 08:07:29+00:00,Unknown,13477941,403,SafeTensorsInfo(parameters={'F32': 2614341888}...,text-generation,NaN,NaN,...,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Gemma2ForCausalLM'],"['transformers', 'safetensors', 'gemma2', 'tex...",license: gemma\nlibrary_name: transformers\npi...,2614341888,1.002944,False,False,True


In [157]:
# # filter models with humaneval_in_card = False and need_permission = False


# # Apply the filter conditionally based on 'need_permission' and 'humaneval_in_card' columns
# filtered_df = filtered_df[
#     ((filtered_df['need_permission']) & (filtered_df['humaneval_in_card'])) | 
#     filtered_df['eval_results'].apply(
#         lambda x: not (isinstance(x, list) and len(x) == 0) and x != "[]" and pd.notna(x)
#     )
# ]

# # Display the filtered DataFrame
# print("Filtered DataFrame Length:", len(filtered_df))
# filtered_df.head()

In [ ]:



# Filter out rows where the 'safetensors' column is NaN
# filtered_df = filtered_df[ 
        
#     filtered_df['eval_results'].apply(
#         lambda x: not (isinstance(x, list) and len(x) == 0) and x != "[]" and pd.notna(x)
#         )
#     or filtered_df['card_data'].str.contains('humaneval', case=False, na=False)
# ]

# filtered_df = filtered_df[
#     filtered_df['card_data'].str.contains('humaneval', case=False, na=False)
    
#     | filtered_df['eval_results'].apply(
#         lambda x: not (isinstance(x, list) and len(x) == 0) and x != "[]" and pd.notna(x)
#     ) 
    
# ]

# print("with eval results or humaneval in card_data:", len(filtered_df))

# print(check_model("google/gemma-2-2b" ,filtered_df))

# import re

# # Define a regex pattern to match 'dataset_name' containing 'humaneval' (case insensitive)
# pattern = r"dataset_name=['\"]humaneval['\"]" # there are other like 'humanevalsynthesis'

# # Filter rows where 'eval_results' matches the regex pattern
# filtered_df = filtered_df[
#     filtered_df['eval_results'].str.contains(pattern, case=False, na=False) |
#     filtered_df['card_data'].str.contains('humaneval', case=False, na=False)
# ]

# print("After humaneval in eval_results: ",len(filtered_df))

# print(check_model("google/gemma-2-2b" ,filtered_df))




with eval results or humaneval in card_data: 1941
False
After humaneval in eval_results:  183
False


In [158]:
save_evaluated_df =True
save_dir = 'model_selection/tmp/'
if save_evaluated_df:
    # Optional: Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(save_dir + "evaluated_models.csv", index=False)
    print(f"Filtered CSV file saved: {save_dir}evaluated_models.csv")

Filtered CSV file saved: model_selection/tmp/evaluated_models.csv


In [160]:
# If filter by any date
filtered_df['created_at'] = pd.to_datetime(filtered_df['created_at'], errors='coerce')

filter_date = False
if filter_date :
    # Filter models with 'created_at' in 2023 or later
    year = 2021
    filtered_df = filtered_df[filtered_df['created_at'].dt.year >= year]

    print(f"filtered w/ created_at len now:{len(filtered_df)}")
else:
    print("Considering all models without date filter")


Considering all models without date filter


In [161]:
# feature: base_model False if it has parent model

filtered_df['base_model'] = ~filtered_df['card_data'].str.contains('base_model', case=False, na=False)


filtered_df[['model','card_data', 'base_model']].head(n=10)



,model,card_data,base_model
3,meta-llama/Llama-3.2-1B,language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,True
5,meta-llama/Llama-3.2-3B-Instruct,language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,True
10,meta-llama/Llama-3.2-1B-Instruct,language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,True
16,meta-llama/Llama-3.2-3B,language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,True
34,google/gemma-2-2b,license: gemma\nlibrary_name: transformers\npi...,True
36,google/gemma-2-2b-it,license: gemma\nlibrary_name: transformers\nta...,False
86,google/gemma-2-2b-jpn-it,language:\n- ja\nlicense: gemma\nlibrary_name:...,False
89,ibm-granite/granite-3.0-2b-instruct,license: apache-2.0\nlibrary_name: transformer...,False
161,meta-llama/Llama-Guard-3-1B,language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,True
163,NousResearch/Llama-3.2-1B,language:\n- en\n- de\n- fr\n- it\n- pt\n- hi\...,True


In [163]:
len(filtered_df)

430

In [162]:
# Count the number of models where 'base_model' is True
base_model_count = filtered_df[filtered_df['base_model'] == True].shape[0]

# Display the count
print("Number of base models == True:", base_model_count)

Number of base models == True: 340


In [164]:

save_filtered_df =True
if save_filtered_df:
    # Optional: Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(save_dir + "filtered_models.csv", index=False)
    print(f"Filtered CSV file saved: {save_dir}filtered_models.csv")

Filtered CSV file saved: model_selection/tmp/filtered_models.csv


In [40]:
# 
import random
i = random.randint(0,len(filtered_df))
print(i)
row = filtered_df.iloc[i]
row

13


model                                                    ibm/PowerLM-3b
id                                                       ibm/PowerLM-3b
created_at                                    2024-08-14 18:20:58+00:00
downloads_all_time                                              Unknown
downloads                                                         16449
likes                                                                17
safetensors           SafeTensorsInfo(parameters={'F32': 3512017152}...
task                                                    text-generation
datasets                                                            NaN
eval_results          [EvalResult(task_type='text-generation', datas...
metrics                                                             NaN
library_name                                               transformers
transformers_info     TransformersInfo(auto_model='AutoModelForCausa...
architecture                                     ['GraniteForCau

# stratified sampling (optional)
- Not done for last experiments due to number of models

In [27]:
save_dir = 'model_selection/results/'
file_name = save_dir+'filtered_models.csv'
df = pd.read_csv(file_name)
print(f'total models: {len(df)}')

total models: 69


In [28]:
df.columns

Index(['model', 'id', 'created_at', 'downloads_all_time', 'downloads', 'likes',
       'safetensors', 'task', 'datasets', 'eval_results', 'metrics',
       'library_name', 'transformers_info', 'architecture', 'tags',
       'card_data', 'num_parameters', 'popularity'],
      dtype='object')

In [29]:
# add likes: popularity = sum of the normalized likes and downloads
stratification_criteria = ['popularity', 'num_parameters','created_at']

In [ ]:
import pandas as pd

# Assuming you have loaded your DataFrame 'df'

# Convert 'created_at' to a numerical format for quartile calculation
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['created_at_numeric'] = df['created_at'].astype(int) // 10**9  # Convert to Unix timestamp, seconds

# Create quartiles for stratified sampling
df['popularity_quartile'] = pd.qcut(df['popularity'], q=2, labels=False)  # 2 quartiles
df['created_at_quartile'] = pd.qcut(df['created_at_numeric'], q=2, labels=False)  # 2 quartiles
df['num_parameters_quartile'] = pd.qcut(df['num_parameters'], q=3, labels=False)  # 3 quartiles



# Display the DataFrame with new quartile columns
print(df[['popularity_quartile', 'created_at_quartile', 'num_parameters_quartile']].head())


   popularity_quartile  created_at_quartile  num_parameters_quartile
0                    1                    1                        1
1                    1                    1                        1
2                    1                    0                        0
3                    1                    0                        0
4                    1                    0                        0


In [33]:
# Group by the quartile columns to create strata
strata = df.groupby(['popularity_quartile', 'created_at_quartile', 'num_parameters_quartile'])

# Select one model from each stratum
selected_models = strata.apply(lambda x: x.sample(1)).reset_index(drop=True)

# Display the selected models

save_selected_df =True 

if save_selected_df:
    # Optional: Save the filtered DataFrame to a CSV file
    selected_models.to_csv(save_dir + "selected_models_01.csv", index=False)
    print(f"Filtered CSV file saved: {save_dir}selected_models.csv")
    
selected_models

Filtered CSV file saved: model_selection/results/selected_models.csv


/tmp/ipykernel_169904/1651244523.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_models = strata.apply(lambda x: x.sample(1)).reset_index(drop=True)


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,...,transformers_info,architecture,tags,card_data,num_parameters,popularity,created_at_numeric,popularity_quartile,created_at_quartile,num_parameters_quartile
0,TheBloke/speechless-code-mistral-7B-v1.0-GPTQ,TheBloke/speechless-code-mistral-7B-v1.0-GPTQ,2023-10-13 06:08:01+00:00,Unknown,19,2,"SafeTensorsInfo(parameters={'I32': 880476160, ...",text-generation,"['jondurbin/airoboros-2.2', 'Open-Orca/OpenOrc...","[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['MistralForCausalLM'],"['transformers', 'safetensors', 'mistral', 'te...",language:\n- en\nlicense: llama2\nlibrary_name...,1198788608,6.558938e-03,1697177281,0,0,0
1,TheBloke/WizardCoder-Python-13B-V1.0-AWQ,TheBloke/WizardCoder-Python-13B-V1.0-AWQ,2023-09-19 00:53:10+00:00,Unknown,22,0,"SafeTensorsInfo(parameters={'I32': 1598361600,...",text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",license: llama2\nlibrary_name: transformers\nt...,2025589760,1.807119e-06,1695084790,0,0,1
2,vodkaslime/test-repo-stablecode,vodkaslime/test-repo-stablecode,2023-08-23 07:50:01+00:00,Unknown,33,0,SafeTensorsInfo(parameters={'BF16': 2769311040...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['GPTNeoXForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'gp...",language:\n- code\nlicense: other\ntags:\n- ca...,3306181952,2.710678e-06,1692777001,0,0,2
3,TechxGenus/starcoder2-3b-GPTQ,TechxGenus/starcoder2-3b-GPTQ,2024-03-22 13:35:25+00:00,Unknown,945,1,"SafeTensorsInfo(parameters={'I32': 363432960, ...",text-generation,['bigcode/the-stack-v2-train'],"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['Starcoder2ForCausalLM'],"['transformers', 'safetensors', 'starcoder2', ...",license: bigcode-openrail-m\nlibrary_name: tra...,688945152,3.356312e-03,1711114525,0,1,0
4,TechxGenus/starcoder2-15b-AWQ,TechxGenus/starcoder2-15b-AWQ,2024-04-12 14:03:22+00:00,Unknown,28,1,"SafeTensorsInfo(parameters={'I32': 1933885440,...",text-generation,['bigcode/the-stack-v2-train'],"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['Starcoder2ForCausalLM'],"['transformers', 'safetensors', 'starcoder2', ...",license: bigcode-openrail-m\nlibrary_name: tra...,2661535744,3.280988e-03,1712930602,0,1,1
5,nlpguy/granite-3.0-3b-a800m-base,nlpguy/granite-3.0-3b-a800m-base,2024-10-31 14:29:25+00:00,Unknown,5,0,SafeTensorsInfo(parameters={'F32': 3374286336}...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['GraniteMoeForCausalLM'],"['transformers', 'safetensors', 'granitemoe', ...",license: apache-2.0\nlibrary_name: transformer...,3374286336,4.107089e-07,1730384965,0,1,2
6,TheBloke/WizardCoder-Python-7B-V1.0-AWQ,TheBloke/WizardCoder-Python-7B-V1.0-AWQ,2023-09-19 00:52:16+00:00,Unknown,118,2,"SafeTensorsInfo(parameters={'I32': 815824896, ...",text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",license: llama2\nlibrary_name: transformers\nt...,1128837120,6.567070e-03,1695084736,1,0,0
7,smallcloudai/Refact-1_6B-fim,smallcloudai/Refact-1_6B-fim,2023-08-29 15:48:36+00:00,Unknown,28033,129,SafeTensorsInfo(parameters={'BF16': 1585842176...,text-generation,"['bigcode/the-stack-dedup', 'rombodawg/2XUNCEN...","[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['GPTRefactForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'gp...",language:\n- en\nlicense: bigscience-op

In [32]:
import pandas as pd

used_models = ["codeparrot/codeparrot-small",
               "EleutherAI/pythia-410m",
               "EleutherAI/pythia-1.4b",
               "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
               "microsoft/phi-2"
               ]

# Assuming 'df' is your DataFrame and you have a specific model ID
model_id = used_models[1]  # Replace with the actual model ID

for model_id in used_models[1:]: 
    # Filter the DataFrame to find the model with the given ID
    model = df[df['id'] == model_id]
    

    # Get the quartile values for the selected model
    popularity_quartile = model['popularity_quartile']
    created_at_quartile = model['created_at_quartile']
    num_parameters_quartile = model['num_parameters_quartile']

    # Display the quartile values
    print("model: ", model['id'])
    print("popularity Quartile:", popularity_quartile)
    print("Created At Quartile:", created_at_quartile)
    print("Num Parameters Quartile:", num_parameters_quartile)
    print("---------------------")


model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Created At Quartile: Series([], Name: created_at_quartile, dtype: int64)
Num Parameters Quartile: Series([], Name: num_parameters_quartile, dtype: int64)
---------------------
model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Created At Quartile: Series([], Name: created_at_quartile, dtype: int64)
Num Parameters Quartile: Series([], Name: num_parameters_quartile, dtype: int64)
---------------------
model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Created At Quartile: Series([], Name: created_at_quartile, dtype: int64)
Num Parameters Quartile: Series([], Name: num_parameters_quartile, dtype: int64)
---------------------
model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Crea